# Imports

In [2]:
! nvidia-smi

Tue Oct 13 03:51:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, Flatten, Dense

import numpy as np
np.random.seed(666)
tf.random.set_seed(666)

In [6]:
# Load the training and testing set of CIFAR10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train = X_train.astype('float32')
X_train = X_train/255.

X_test = X_test.astype('float32')
X_test = X_test/255.

y_train = tf.reshape(tf.one_hot(y_train, 10), shape=(-1, 10))
y_test = tf.reshape(tf.one_hot(y_test, 10), shape=(-1, 10))

170500096/170498071 [==============================] - 4s 0us/step


In [7]:
# Create TensorFlow dataset
BATCH_SIZE = 256
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(1024).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [8]:
for image, label in train_ds.take(1):
  print(image.shape)
  print(label.shape)

(256, 32, 32, 3)
(256, 10)


In [29]:
class Organism:
    def __init__(self, chromosome={}, phase=0):
      self.phase = phase
      self.chromosome = chromosome
    
    def build_model(self):
      if self.phase == 0:
        keras.backend.clear_session()
        
        inputs = Input(shape=(32,32,3))
        x = Conv2D(filters=self.chromosome['a_output_channels'],
                    kernel_size=self.chromosome['a_filter_size'],
                    use_bias=self.chromosome['a_include_BN'])(inputs)
        if self.chromosome['a_include_BN']:
            x = BatchNormalization()(x)
        x = self.chromosome['activation_type']()(x)
        if self.chromosome['include_pool']:
            x = self.chromosome['pool_type']()(x)

        x = Conv2D(filters=self.chromosome['b_output_channels'],
                    kernel_size=self.chromosome['b_filter_size'],
                    use_bias=self.chromosome['b_include_BN'])(x)
        if self.chromosome['b_include_BN']:
            x = BatchNormalization()(x)
        x = self.chromosome['activation_type']()(x)

        x = Flatten()(x)
        x = Dense(10, activation='softmax')(x)

        self.model = tf.keras.Model(inputs=[inputs], outputs=[x])
        self.model.compile(optimizer='adam',
                            loss='categorical_crossentropy',
                            metrics=['accuracy'])
      else:
        print('Phase under construction')
    
    def fitnessFunction(self, train_ds, test_ds):
      self.model.fit(train_ds,
                      epochs=5,
                      verbose=1)
      _, self.fitness = self.model.evaluate(test_ds, verbose=0)
    
    def crossover(self, partner):
      child = Organism(chromosome={}, phase=0)
      endpoint = np.random.randint(low=0, high=len(self.chromosome))
      for idx, key in enumerate(self.chromosome):
        if idx <= endpoint:
          child.chromosome[key] = self.chromosome[key]
        else:
          child.chromosome[key] = partner.chromosome[key]
      return child
    
    def mutation(self, mutationRate):
        pass

In [30]:
org1 = Organism({'a_output_channels':32,
                 'b_output_channels':18,
                 'a_filter_size':(5,5),
                 'b_filter_size':(5,5),
                 'a_include_BN':False,
                 'b_include_BN':True,
                 'activation_type': ReLU,
                 'include_pool': True,
                 'pool_type': MaxPool2D
                 }, 
                phase=0)

org2 = Organism({'a_output_channels':64,
                 'b_output_channels':128,
                 'a_filter_size':(3,3),
                 'b_filter_size':(3,3),
                 'a_include_BN':True,
                 'b_include_BN':True,
                 'activation_type': ReLU,
                 'include_pool': True,
                 'pool_type': MaxPool2D
                 }, 
                phase=0)

org1.build_model()
org2.build_model()

In [14]:
org1.fitnessFunction(train_ds, test_ds)
org2.fitnessFunction(train_ds, test_ds)

Epoch 1/5
196/196 [==============================] - 1s 6ms/step - loss: 1.4837 - accuracy: 0.4651
Epoch 2/5
196/196 [==============================] - 1s 6ms/step - loss: 1.1622 - accuracy: 0.5915
Epoch 3/5
196/196 [==============================] - 1s 5ms/step - loss: 1.0519 - accuracy: 0.6288
Epoch 4/5
196/196 [==============================] - 1s 5ms/step - loss: 0.9833 - accuracy: 0.6566
Epoch 5/5
196/196 [==============================] - 1s 5ms/step - loss: 0.9329 - accuracy: 0.6747
Epoch 1/5
196/196 [==============================] - 2s 11ms/step - loss: 1.6842 - accuracy: 0.4975
Epoch 2/5
196/196 [==============================] - 2s 11ms/step - loss: 1.0400 - accuracy: 0.6489
Epoch 3/5
196/196 [==============================] - 2s 11ms/step - loss: 0.8689 - accuracy: 0.7070
Epoch 4/5
196/196 [==============================] - 2s 11ms/step - loss: 0.7501 - accuracy: 0.7447
Epoch 5/5
196/196 [==============================] - 2s 11ms/step - loss: 0.6675 - accuracy: 0.7701


In [15]:
print('org1: {:0.2f}'.format(org1.fitness))
print('org2: {:0.2f}'.format(org2.fitness))

org1: 0.63
org2: 0.61


In [31]:
org1_2 = org1.crossover(org2)
org1_2.chromosome

{'a_filter_size': (3, 3),
 'a_include_BN': True,
 'a_output_channels': 32,
 'activation_type': tensorflow.python.keras.layers.advanced_activations.ReLU,
 'b_filter_size': (3, 3),
 'b_include_BN': True,
 'b_output_channels': 18,
 'include_pool': True,
 'pool_type': tensorflow.python.keras.layers.pooling.MaxPooling2D}

In [32]:
org1_2.build_model()
org1_2.fitnessFunction(train_ds, test_ds)

Epoch 1/5
196/196 [==============================] - 1s 7ms/step - loss: 1.4952 - accuracy: 0.4648
Epoch 2/5
196/196 [==============================] - 1s 7ms/step - loss: 1.1518 - accuracy: 0.5940
Epoch 3/5
196/196 [==============================] - 1s 7ms/step - loss: 1.0167 - accuracy: 0.6454
Epoch 4/5
196/196 [==============================] - 1s 7ms/step - loss: 0.9357 - accuracy: 0.6744
Epoch 5/5
196/196 [==============================] - 1s 7ms/step - loss: 0.8813 - accuracy: 0.6952


In [34]:
print('org1_2: {:0.2f}'.format(org1_2.fitness))

org1_2: 0.64
